In [27]:
import io
import pandas as pd
from pprint import pprint
import requests
from urllib.parse import quote

def ades_GetPointEau(numéro_département=None, nb_réseau=None):
    url = 'https://ades.eaufrance.fr/Recherche/GetPointEau'
    query_params = {
        'type': '1',
        'filter': quote(f'departement={numéro_département}&nbReseau={nb_réseau}'),
        'rows': '500',
        'start': '0',
        'statut_qualif': '0'
    }
    response_GetPointEau = requests.post(url=url, data=query_params)
    return response_GetPointEau.json()['pointEaux']
    

def ades_GetPointEauIPSDataCSV(numéro_département=None, nombre_points=None):
    url = 'https://ades.eaufrance.fr/Recherche/GetPointEauIPSDataCSV'
    query_params = {
        'rows': -1,
        'start': 0,
        'filters': quote(f'departement={numéro_département}'),
        'nombre_points': nombre_points,
        'typeRecherche': 1,
        'date_debut': None,
        'date_fin': None,
        'moisAnnee': None
    }
    response_GetPointEauIPSDataCSV = requests.get(url=url, params=query_params)
    return response_GetPointEauIPSDataCSV.content
    
    
numéro_département = 66
point_eau_list = ades_GetPointEau(numéro_département, nb_réseau=0)
print(f"\n\nExtrait des données de l'API https://ades.eaufrance.fr/Recherche/GetPointEau pour le département {numéro_département} :\n\n")
pprint(point_eau_list[0])
point_eau_ips_data_csv_bytes = ades_GetPointEauIPSDataCSV(numéro_département, nombre_points=len(point_eau_list))
df_point_eau_ips_data = pd.read_csv(io.BytesIO(point_eau_ips_data_csv_bytes), sep=';')
print(f"\n\nExtrait des données de l'API https://ades.eaufrance.fr/Recherche/GetPointEauIPSDataCSV pour le département {numéro_département} :\n\n")
df_point_eau_ips_data



Extrait des données de l'API https://ades.eaufrance.fr/Recherche/GetPointEau pour le département 66 :


{'Coordinates': ['2.83846783706838', '42.68520376443'],
 'Properties': {'coord_com': 'N',
                'coord_ref': ['686745.1381916221', '6176189.532351297'],
                'epsg_ref': 2154,
                'exist_mesures': None,
                'label': 'BSS002MPSM (10908X0263/FIGUER)',
                'position_captage': None,
                'type_pt_eau': '1'},
 'Type': 'Point'}


Extrait des données de l'API https://ades.eaufrance.fr/Recherche/GetPointEauIPSDataCSV pour le département 66 :




,Identifiant BSS,Ancien code national BSS,Type de suivi,Commune,Département,Nb de mesures,Date de début,Date de fin,Nb de réseaux
0,BSS002MHQQ,10795X0001/S,Piézomètre et qualitomètre,Salses-le-Château (66190),Pyrénées-Orientales (66),3034,27/02/2002,11/01/2011,9
1,BSS002MHRT,10795X0028/CARSTE,Piézomètre,Salses-le-Château (66190),Pyrénées-Orientales (66),9029,09/12/1997,06/06/2023,7
2,BSS002MHTK,10795X0068/S107,Piézomètre,Salses-le-Château (66190),Pyrénées-Orientales (66),7082,05/06/2002,05/06/2023,1
3,BSS002MHTM,10795X0070/PZCOMB,Piézomètre,Salses-le-Château (66190),Pyrénées-Orientales (66),5736,07/03/2005,04/06/2023,1
4,BSS002MNDU,10894X0013/RESURG,Piézomètre et qualitomètre,Saint-Paul-de-Fenouillet (66187),Pyrénées-Orientales (66),6869,22/02/2002,06/06/2023,13
...,...,...,...,...,...,...,...,...,...
61,BSS002MVSY,10988X0001/FAITOU,Piézomètre et qualitomètre,Valcebollère (66220),Pyrénées-Orientales (66),3995,28/06/2012,06/06/2023,3
62,BSS002MVXJ,11001X0005/CECILE,Piézomètre et qualitomètre,Le Tech (66206),Pyrénées-Orientales (66),5333,20/10/2008,06/06/2023,7
63,BSS003FATM,N/D,Piézomètre,Trouillas (66217),Pyrénées-Orientales (66),1563,11/12/2018,02/06/2023,1
64,BSS003FAXY,N/D,Piézomètre,Rivesaltes (66164),Pyrénées-Orientales (66),1551,11/12/2018,04/06/2023,1
